In [1]:
import json
import requests
import warnings


# pybliometrics

Enter your API key(s)

In [2]:
from pybliometrics import scopus

# Needs input for the first run, to create the config file

scopus.utils.constants.CONFIG_FILE

print(scopus.utils.config['Authentication']['APIKey'])

Creating config file at C:\Users\AliApg\.config\pybliometrics.cfg with default paths...


Configuration file successfully created at C:\Users\AliApg\.config\pybliometrics.cfg
For details see https://pybliometrics.rtfd.io/en/stable/configuration.html.
5227c64c7de74597d4c2c75d6eb5bc16


In [3]:
try:
    author_results = scopus.AuthorRetrieval(7004212771)
    print(author_results)

except Exception as err:
    warnings.warn(str(err))


C:\Users\AliApg\AppData\Local\Temp\ipykernel_68952\3988203531.py:6: UserWarning: The requestor is not authorized to access the requested view or fields of the resource
  warnings.warn(str(err))


In [35]:
try:
    # results = scopus.ScopusSearch("KEY(nlp)")
    results = scopus.ScopusSearch("NLP", subscriber=False)

    print(f'{results.get_results_size()} results were found!\n\n',results, sep='')

except Exception as err:
    warnings.warn(str(err))


C:\Users\AliApg\AppData\Local\Temp\ipykernel_68952\1810647757.py:8: UserWarning: Found 96,651 matches.  The query fails to return more than 5000 entries.  Change your query such that it returns fewer entries.
  warnings.warn(str(err))


In [26]:
a = '''2-s2.0-85129252816
2-s2.0-85162105000
2-s2.0-85154025314
2-s2.0-85152003122
2-s2.0-85151146663
2-s2.0-85134911178
2-s2.0-85132821689
2-s2.0-85130418556
2-s2.0-85122805619
2-s2.0-85123435934
2-s2.0-85148419013
2-s2.0-85113431172
2-s2.0-85119472934
2-s2.0-85108690885
2-s2.0-85072528498
2-s2.0-85126716062
2-s2.0-85125851265
2-s2.0-85124008870
2-s2.0-85103509107
2-s2.0-85101779605
2-s2.0-85095864117
2-s2.0-85084481741
2-s2.0-85098232302
2-s2.0-85082009615
2-s2.0-85077293806
2-s2.0-85077458726
2-s2.0-85070876987
2-s2.0-85064582151
2-s2.0-85047401887
2-s2.0-85046349764
2-s2.0-85029379569'''.split('\n')

a = '''2-s2.0-85129252816
2-s2.0-85154025314
2-s2.0-85130418556
2-s2.0-85125851265
2-s2.0-85095864117
2-s2.0-85084481741
2-s2.0-85098232302
2-s2.0-85077293806
2-s2.0-85070876987
2-s2.0-85046349764'''.split('\n')

b = '''2-s2.0-85129252816
2-s2.0-85162105000
2-s2.0-85154025314
2-s2.0-85152003122
2-s2.0-85134911178
2-s2.0-85132821689
2-s2.0-85130418556
2-s2.0-85123435934
2-s2.0-85148419013
2-s2.0-85113431172
2-s2.0-85108690885
2-s2.0-85072528498
2-s2.0-85126716062
2-s2.0-85125851265
2-s2.0-85103509107
2-s2.0-85084481741
2-s2.0-85098232302
2-s2.0-85082009615
2-s2.0-85077293806
2-s2.0-85070876987
2-s2.0-85128492821
2-s2.0-85029379569'''.split()

for al in a:
    bl = [True for by in b if al==by]
    print(bl)

[True]
[True]
[True]
[True]
[]
[True]
[True]
[True]
[True]
[]


# NO lib

In [5]:
# API_KEY = '0a8e79cfa41aeded29d60f49ab30d67c'  # API key 1
API_KEY = '5227c64c7de74597d4c2c75d6eb5bc16'    # API key 2

# API_KEY = '2486a63b5d616aa4de0576feb95e99a5'  # your API key


In [8]:
scopus_id = 'SCOPUS_ID:85153301600'

resp = requests.get("http://api.elsevier.com/content/author?author_id=7004212771&view=metrics",
# resp = requests.get(f"https://api.elsevier.com/content/abstract/citation-count/{scopus_id}",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': API_KEY})

print(json.dumps(resp.json(),
                #  sort_keys=True,
                 indent=4, separators=(',', ': ')))


{
    "service-error": {
        "status": {
            "statusCode": "AUTHORIZATION_ERROR",
            "statusText": "The requestor is not authorized to access the requested view or fields of the resource"
        }
    }
}


In [7]:
def get_scopus_info(SCOPUS_ID):
    url = f"http://api.elsevier.com/content/abstract/scopus_id/{SCOPUS_ID}"\
    "?field=authors,title,publicationName,volume,issueIdentifier,prism:pageRange"\
    ",coverDate,article-number,doi,citedby-count,prism:aggregationType"

    resp = requests.get(url,
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': API_KEY})

    # print(json.dumps(resp.json(),
    #              sort_keys=True,
    #              indent=4, separators=(',', ': ')))

    results = json.loads(resp.text.encode('utf-8'))

    ARR = 'abstracts-retrieval-response'
    try:
        authors=', '.join([au['ce:indexed-name']
                           for au in results[ARR]['authors']['author']])
    except Exception:
        authors= ''

    title=results[ARR]['coredata']['dc:title']
    journal=results[ARR]['coredata']['prism:publicationName']
    volume=results[ARR]['coredata']['prism:volume']
    articlenum=(results[ARR]['coredata'].get('prism:pageRange') or
                results[ARR]['coredata'].get('article-number'))
    date=results[ARR]['coredata']['prism:coverDate']
    doi=f"doi:{results[ARR]['coredata']['prism:doi']}"
    cites=int(results[ARR]['coredata']['citedby-count'])

    return f'{authors},\n{title},\nJournal: {journal}, Volume: {volume}, '\
    f'Article number: {articlenum}, ({date}).\n{doi} (cited {cites} times).\n'

print(get_scopus_info('SCOPUS_ID:0037368024'))

,
A comparison of gold and molybdenum nanoparticles on TiO2(1 1 0) 1 × 2 reconstructed single crystal surfaces,
Journal: Surface Science, Volume: 526, Article number: 323-331, (2003-03-01).
doi:10.1016/S0039-6028(02)02679-1 (cited 33 times).

